In [5]:
import struct
import numpy as np
import matplotlib.pyplot as plt

def parse_idx3(path):
    # read bin data from file
    bin_data = open(path, 'rb').read()

    # parse head info
    offset = 0
    # read first 4 line
    format_head = '>iiii'
    magic_number, item_num, row_num, col_num = struct.unpack_from(format_head, bin_data, offset)
    print('magic number: %d' % magic_number)
    print('item num: %d, %d row * %d col' % (item_num, row_num, col_num))

    # parse data
    item_size = row_num * col_num
    # read one item once
    format_data = '>' + str(item_size) + 'B'
    # set offest to where we have finished reading
    offset = struct.calcsize(format_head)
    # create an empty array and fill it
    items = np.empty((item_num, row_num, col_num))
    for i in range(item_num):
        items[i] = np.array(struct.unpack_from(format_data, bin_data, offset)).reshape((row_num, col_num))
        offset = offset + struct.calcsize(format_data)
    return items

# same as parse_idx3
def parse_idx1(path):
    bin_data = open(path, 'rb').read()
    offset = 0
    format_head = '>ii'
    magic_number, item_num = struct.unpack_from(format_head, bin_data, offset)
    print('magic number: %d' % magic_number)
    print('item num: %d' % item_num)
    format_data = '>B'
    offset = struct.calcsize(format_head)
    items = np.empty(item_num)
    for i in range(item_num):
        items[i] = struct.unpack_from(format_data, bin_data, offset)[0]
        offset = offset + struct.calcsize(format_data)
    return items


train_labels_path = 'D:/MY/ml-data/handwritting-recognition/train-labels.idx1-ubyte'
train_images_path = 'D:/MY/ml-data/handwritting-recognition/train-images.idx3-ubyte'

# show first label and image
train_labels = parse_idx1(train_labels_path)
train_images = parse_idx3(train_images_path)
print(train_labels[0])
plt.imshow(train_images[0], cmap='gray')
plt.show()

magic number: 2049
item num: 60000
magic number: 2051
item num: 60000, 28 row * 28 col
[5. 0. 4. ... 5. 6. 8.]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0.

In [6]:
from keras.utils import to_categorical

# read from file
train_labels_path = 'D:/MY/ml-data/handwritting-recognition/train-labels.idx1-ubyte'
train_images_path = 'D:/MY/ml-data/handwritting-recognition/train-images.idx3-ubyte'
train_labels = parse_idx1(train_labels_path)
train_images - parse_idx3(train_images_path)

# format array
train_labels = to_categorical(train_labels, 10)
train_images = train_images.reshape(-1, 28, 28, 1)
train_images = train_images.astype('float32')
train_images /= 255

# build model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=[28, 28, 1]))
model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=categorical_crossentropy,
             optimizer=Adadelta(),
             metrics=['accuracy'])

batch_size = 100
epochs = 8
model.fit(train_images, train_labels,
         batch_size=batch_size,
         epochs=epochs)

# load test data and test accuracy
train_labels_path = 'D:/MY/ml-data/handwritting-recognition/train-labels.idx1-ubyte'
train_images_path = 'D:/MY/ml-data/handwritting-recognition/train-images.idx3-ubyte'
test_X, test_y = mnist.load_data()[1]
test_X = test_X.reshape(-1, 28, 28, 1)
test_X = test_X.astype('float32')
test_X /= 255
test_y = to_categorical(test_y, 10)
loss, accuracy = model.evaluate(test_X, test_y, verbose=1)
print('loss:%.4f accuracy:%.4f' %(loss, accuracy))

[5 0 4 ... 5 6 8]
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
